# 如何 dispatch 自定义回调事件

:::info 先决条件

本指南假定您熟悉以下概念：

- [回调](/docs/concepts/callbacks)
- [自定义回调处理器](/docs/how_to/custom_callbacks)
- [Astream Events API](/docs/concepts/streaming/#astream_events) `astream_events` 方法将暴露自定义回调事件。
:::

在某些情况下，您可能希望在 [Runnable](/docs/concepts/runnables) 内部 dispatch 一个自定义回调事件，以便它可以被自定义回调处理器或 [Astream Events API](/docs/concepts/streaming/#astream_events) 所暴露。

例如，如果您有一个包含多个步骤的长时间运行的工具，您可以在步骤之间 dispatch 自定义事件，并使用这些自定义事件来监控进度。您也可以将这些自定义事件暴露给应用程序的最终用户，向他们展示当前任务的进展情况。

要 dispatch 自定义事件，您需要确定事件的两个属性：`name` 和 `data`。

| 属性 | 类型 | 描述                                                                                             |
|---|---|--------------------------------------------------------------------------------------------------|
| name | str | 用户定义的事件名称。                                                                           |
| data | Any | 与事件关联的数据。这可以是任何内容，但我们建议使其可 JSON 序列化。                                                               |


:::important
* Dispatching 自定义回调事件需要 `langchain-core>=0.2.15`。
* 自定义回调事件只能在现有的 `Runnable` 内部 dispatch。
* 如果使用 `astream_events`，则必须使用 `version='v2'` 才能看到自定义事件。
* LangSmith 中发送或渲染自定义回调事件尚不支持。
:::


:::caution 兼容性
如果您在 `python<=3.10` 中运行异步代码，LangChain 无法自动将配置（包括 astream_events() 所必需的回调）传播到子 Runnable。这是您可能无法看到从自定义 Runnable 或工具发出的事件的一个常见原因。

如果您运行的是 `python<=3.10`，您需要在异步环境中手动将 `RunnableConfig` 对象传播到子 Runnable。有关如何手动传播配置的示例，请参阅下面 `bar` RunnableLambda 的实现。

如果您运行的是 `python>=3.11`，`RunnableConfig` 将会自动传播到异步环境中的子 Runnable。但是，如果您的代码可能在其他 Python 版本上运行，手动传播 `RunnableConfig` 仍然是一个好主意。
:::

In [ ]:
# | output: false
# | echo: false

%pip install -qU langchain-core

## Astream 事件 API

消费自定义事件最有用的方式是使用 [Astream 事件 API](/docs/concepts/streaming/#astream_events)。

我们可以使用 `async` 的 `adispatch_custom_event` API 在异步环境中触发自定义事件。

:::important

要通过 Astream 事件 API 查看自定义事件，您需要使用 `astream_events` 的较新 `v2` API。
:::

In [3]:
from langchain_core.callbacks.manager import (
    adispatch_custom_event,
)
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.config import RunnableConfig


@RunnableLambda
async def foo(x: str) -> str:
    await adispatch_custom_event("event1", {"x": x})
    await adispatch_custom_event("event2", 5)
    return x


async for event in foo.astream_events("hello world", version="v2"):
    print(event)

{'event': 'on_chain_start', 'data': {'input': 'hello world'}, 'name': 'foo', 'tags': [], 'run_id': 'f354ffe8-4c22-4881-890a-c1cad038a9a6', 'metadata': {}, 'parent_ids': []}
{'event': 'on_custom_event', 'run_id': 'f354ffe8-4c22-4881-890a-c1cad038a9a6', 'name': 'event1', 'tags': [], 'metadata': {}, 'data': {'x': 'hello world'}, 'parent_ids': []}
{'event': 'on_custom_event', 'run_id': 'f354ffe8-4c22-4881-890a-c1cad038a9a6', 'name': 'event2', 'tags': [], 'metadata': {}, 'data': 5, 'parent_ids': []}
{'event': 'on_chain_stream', 'run_id': 'f354ffe8-4c22-4881-890a-c1cad038a9a6', 'name': 'foo', 'tags': [], 'metadata': {}, 'data': {'chunk': 'hello world'}, 'parent_ids': []}
{'event': 'on_chain_end', 'data': {'output': 'hello world'}, 'run_id': 'f354ffe8-4c22-4881-890a-c1cad038a9a6', 'name': 'foo', 'tags': [], 'metadata': {}, 'parent_ids': []}


在 Python 3.10 及之前的版本中，您必须手动传播配置！

In [7]:
from langchain_core.callbacks.manager import (
    adispatch_custom_event,
)
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.config import RunnableConfig


@RunnableLambda
async def bar(x: str, config: RunnableConfig) -> str:
    """An example that shows how to manually propagate config.

    You must do this if you're running python<=3.10.
    """
    await adispatch_custom_event("event1", {"x": x}, config=config)
    await adispatch_custom_event("event2", 5, config=config)
    return x


async for event in bar.astream_events("hello world", version="v2"):
    print(event)

{'event': 'on_chain_start', 'data': {'input': 'hello world'}, 'name': 'bar', 'tags': [], 'run_id': 'c787b09d-698a-41b9-8290-92aaa656f3e7', 'metadata': {}, 'parent_ids': []}
{'event': 'on_custom_event', 'run_id': 'c787b09d-698a-41b9-8290-92aaa656f3e7', 'name': 'event1', 'tags': [], 'metadata': {}, 'data': {'x': 'hello world'}, 'parent_ids': []}
{'event': 'on_custom_event', 'run_id': 'c787b09d-698a-41b9-8290-92aaa656f3e7', 'name': 'event2', 'tags': [], 'metadata': {}, 'data': 5, 'parent_ids': []}
{'event': 'on_chain_stream', 'run_id': 'c787b09d-698a-41b9-8290-92aaa656f3e7', 'name': 'bar', 'tags': [], 'metadata': {}, 'data': {'chunk': 'hello world'}, 'parent_ids': []}
{'event': 'on_chain_end', 'data': {'output': 'hello world'}, 'run_id': 'c787b09d-698a-41b9-8290-92aaa656f3e7', 'name': 'bar', 'tags': [], 'metadata': {}, 'parent_ids': []}


## 异步回调处理程序

您也可以通过异步回调处理程序来使用已分派的事件。

In [8]:
from typing import Any, Dict, List, Optional
from uuid import UUID

from langchain_core.callbacks import AsyncCallbackHandler
from langchain_core.callbacks.manager import (
    adispatch_custom_event,
)
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.config import RunnableConfig


class AsyncCustomCallbackHandler(AsyncCallbackHandler):
    async def on_custom_event(
        self,
        name: str,
        data: Any,
        *,
        run_id: UUID,
        tags: Optional[List[str]] = None,
        metadata: Optional[Dict[str, Any]] = None,
        **kwargs: Any,
    ) -> None:
        print(
            f"Received event {name} with data: {data}, with tags: {tags}, with metadata: {metadata} and run_id: {run_id}"
        )


@RunnableLambda
async def bar(x: str, config: RunnableConfig) -> str:
    """An example that shows how to manually propagate config.

    You must do this if you're running python<=3.10.
    """
    await adispatch_custom_event("event1", {"x": x}, config=config)
    await adispatch_custom_event("event2", 5, config=config)
    return x


async_handler = AsyncCustomCallbackHandler()
await foo.ainvoke(1, {"callbacks": [async_handler], "tags": ["foo", "bar"]})

Received event event1 with data: {'x': 1}, with tags: ['foo', 'bar'], with metadata: {} and run_id: a62b84be-7afd-4829-9947-7165df1f37d9
Received event event2 with data: 5, with tags: ['foo', 'bar'], with metadata: {} and run_id: a62b84be-7afd-4829-9947-7165df1f37d9


1

## 同步回调处理程序

让我们看看如何使用 `dispatch_custom_event` 在同步环境中触发自定义事件。

你**必须**在现有的 `Runnable` 中调用 `dispatch_custom_event`。

In [5]:
from typing import Any, Dict, List, Optional
from uuid import UUID

from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.callbacks.manager import (
    dispatch_custom_event,
)
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.config import RunnableConfig


class CustomHandler(BaseCallbackHandler):
    def on_custom_event(
        self,
        name: str,
        data: Any,
        *,
        run_id: UUID,
        tags: Optional[List[str]] = None,
        metadata: Optional[Dict[str, Any]] = None,
        **kwargs: Any,
    ) -> None:
        print(
            f"Received event {name} with data: {data}, with tags: {tags}, with metadata: {metadata} and run_id: {run_id}"
        )


@RunnableLambda
def foo(x: int, config: RunnableConfig) -> int:
    dispatch_custom_event("event1", {"x": x})
    dispatch_custom_event("event2", {"x": x})
    return x


handler = CustomHandler()
foo.invoke(1, {"callbacks": [handler], "tags": ["foo", "bar"]})

Received event event1 with data: {'x': 1}, with tags: ['foo', 'bar'], with metadata: {} and run_id: 27b5ce33-dc26-4b34-92dd-08a89cb22268
Received event event2 with data: {'x': 1}, with tags: ['foo', 'bar'], with metadata: {} and run_id: 27b5ce33-dc26-4b34-92dd-08a89cb22268


1

## 后续步骤

你已经了解了如何发出自定义事件，可以查看更深入的指南 [流式事件](/docs/how_to/streaming/#using-stream-events)，这是利用自定义事件最简单的方法。